Referance/Copied from for my personal referance 

https://www.geeksforgeeks.org/implement-your-own-word2vecskip-gram-model-in-python/

https://towardsdatascience.com/an-implementation-guide-to-word2vec-using-numpy-and-google-sheets-13445eebd281
    
    

In [1]:
import numpy as np 
import string 
from nltk.corpus import stopwords  

In [2]:
def softmax(x): 
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x)) 
    return e_x / e_x.sum() 
   

In [3]:
class word2vec(object): 
    def __init__(self): 
        self.N = 10
        self.X_train = [] 
        self.y_train = [] 
        self.window_size = 2
        self.alpha = 0.001
        self.words = [] 
        self.word_index = {} 
   
    def initialize(self,V,data): 
        self.V = V 
        self.W = np.random.uniform(-0.8, 0.8, (self.V, self.N)) 
        self.W1 = np.random.uniform(-0.8, 0.8, (self.N, self.V)) 
           
        self.words = data 
        for i in range(len(data)): 
            self.word_index[data[i]] = i 
   
       
    def feed_forward(self,X): 
        self.h = np.dot(self.W.T,X).reshape(self.N,1) 
        self.u = np.dot(self.W1.T,self.h) 
        #print(self.u) 
        self.y = softmax(self.u)   
        #print(self.y)
        return self.y 
           
    def backpropagate(self,x,t): 
        e = self.y - np.asarray(t).reshape(self.V,1) 
        # e.shape is V x 1 
        dLdW1 = np.dot(self.h,e.T) 
        X = np.array(x).reshape(self.V,1) 
        dLdW = np.dot(X, np.dot(self.W1,e).T) 
        self.W1 = self.W1 - self.alpha*dLdW1 
        self.W = self.W - self.alpha*dLdW 
           
    def train(self,epochs): 
        for x in range(1,epochs):         
            self.loss = 0
            for j in range(len(self.X_train)): 
                self.feed_forward(self.X_train[j]) 
                self.backpropagate(self.X_train[j],self.y_train[j]) 
                C = 0
                for m in range(self.V): 
                    if(self.y_train[j][m]): 
                        self.loss += -1*self.u[m][0] 
                        #print(m,  "---m")
                        C += 1
                self.loss += C*np.log(np.sum(np.exp(self.u))) 
            print("epoch ",x, " loss = ",self.loss) 
            self.alpha *= 1/( (1+self.alpha*x) ) 
        print(self.W)
              
    def predict(self,word,number_of_predictions): 
        if word in self.words: 
            index = self.word_index[word] 
            X = [0 for i in range(self.V)] 
            X[index] = 1
            prediction = self.feed_forward(X) 
            output = {} 
            for i in range(self.V): 
                output[prediction[i][0]] = i 
               
            top_context_words = [] 
            for k in sorted(output,reverse=True): 
                top_context_words.append(self.words[output[k]]) 
                if(len(top_context_words)>=number_of_predictions): 
                    break
       
            return top_context_words 
        else: 
            print("Word not found in dicitonary")

In [4]:
def preprocessing(corpus): 
    stop_words = set(stopwords.words('english'))     
    training_data = [] 
    sentences = corpus.split(".") 
    for i in range(len(sentences)): 
        sentences[i] = sentences[i].strip() 
        sentence = sentences[i].split() 
        x = [word.strip(string.punctuation) for word in sentence 
                                     if word not in stop_words] 
        x = [word.lower() for word in x] 
        training_data.append(x) 
    return training_data 
       
def prepare_data_for_training(sentences,w2v): 
    data = {} 
    for sentence in sentences: 
        for word in sentence: 
            if word not in data: 
                data[word] = 1
            else: 
                data[word] += 1
    
    V = len(data) 
    data = sorted(list(data.keys())) 

    vocab = {} 
    for i in range(len(data)): 
        vocab[data[i]] = i 
    
    for sentence in sentences: 
        for i in range(len(sentence)): 
            center_word = [0 for x in range(V)] 
            center_word[vocab[sentence[i]]] = 1
            
            context = [0 for x in range(V)] 
                          
            for j in range(i-w2v.window_size,i+w2v.window_size): 
                
                if i!=j and j>=0 and j<len(sentence): 
                    context[vocab[sentence[j]]] += 1
            w2v.X_train.append(center_word) 
            w2v.y_train.append(context) 
            
    w2v.initialize(V,data) 
    
   
    return w2v.X_train,w2v.y_train   

In [9]:
corpus = "" 
corpus += "Prime Minister Narendra Modi was accorded a warm welcome by members of the Indian community in Houston on September 21 as he arrived for the mega Howdy Modi event in which he will be joined by U.S. President Donald Trump and address over 50,000 Indian-Americans.On the way to Houston, Mr. Modi’s flight had a two-hour technical halt in Frankfurt early on September 21 where he was received by India’s Ambassador to Germany Mukta Tomar and Consul General Pratibha Parkar.The Howdy Modi event on September 22 at the sprawling NRG Football Stadium in Houston is the largest gathering ever for an elected foreign leader visiting the U.S., other than the Pope.Mr. Trump’s presence at the Houston event marks a new milestone, Mr. Modi had said in his departure statement ahead of his visit. This would be the first time a U.S. President is attending an Indian-American community event with Modi.In Houston, Mr. Modi will also interact with CEOs of leading American energy companies with an aim to enhance India-US energy partnership.From Houston, Mr. Modi will fly to New York where he will address the annual high-level UN General Assembly session in New York on September 27 and will have a packed agenda of bilateral and multilateral engagements.Mr. Modi will met Mr. Trump in New York on September 24, the fourth meeting between the two leaders in as many months. The meeting in New York is expected to set the tone of bilateral relationship between the two countries over the next few years.The two leaders are likely to discuss a range of bilateral, regional and global issues, including efforts to address the growing bilateral trade disputes, potential defence and energy deals and peace process in Afghanistan.In his departure statement, Mr. Modi referred to the Indo-U,S. relations and said working together, the two nations can contribute to building a more peaceful, stable, secure, sustainable and prosperous world.Mr. Modi also said that at the High Level Segment of the 74th Session of the United Nations General Assembly, he would reiterate New Delhi’s commitment to reformed multilateralism, which is responsive, effective and inclusive, and in which India plays her due role.Foreign Secretary Vijay Gokhale in New Delhi on September 19 said the focus of Mr. Modi’s week-long visit to the U.S. to attend the UNGA will not be on terrorism, but on highlighting India’s achievements and its global role.He also asserted that abrogation of provisions of Article 370 from Jammu and Kashmir was an internal issue and off the agenda at the UN."
epochs = 100
  
training_data = preprocessing(corpus) 
w2v = word2vec() 
  
prepare_data_for_training(training_data,w2v) 
w2v.train(epochs)  
  
print(w2v.predict("welcome",3))   


epoch  1  loss =  3843.339655409778
epoch  2  loss =  3839.553637343515
epoch  3  loss =  3835.785623493039
epoch  4  loss =  3832.03895924129
epoch  5  loss =  3828.3168978205545
epoch  6  loss =  3824.622583320327
epoch  7  loss =  3820.959035030192
epoch  8  loss =  3817.3291333049765
epoch  9  loss =  3813.7356071040363
epoch  10  loss =  3810.181023318409
epoch  11  loss =  3806.667777960095
epoch  12  loss =  3803.1980892487686
epoch  13  loss =  3799.7739925931
epoch  14  loss =  3796.3973374291304
epoch  15  loss =  3793.069785845838
epoch  16  loss =  3789.79281290088
epoch  17  loss =  3786.5677085062503
epoch  18  loss =  3783.3955807457655
epoch  19  loss =  3780.2773604730564
epoch  20  loss =  3777.213807030447
epoch  21  loss =  3774.2055149254593
epoch  22  loss =  3771.25292130164
epoch  23  loss =  3768.3563140444426
epoch  24  loss =  3765.515840369649
epoch  25  loss =  3762.731515750748
epoch  26  loss =  3760.0032330530526
epoch  27  loss =  3757.3307717542384
epo